In [1]:
# https://www.kaggle.com/del=59c8c55006eb2533/nips-papers-visualized-with-nmf-and-t-sne
import pandas as pd
import re
from datetime import datetime
from imblearn.over_sampling import SMOTE

from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

from gensim.summarization.summarizer import summarize
import warnings
warnings.filterwarnings("ignore")
import json
import urllib
from bs4 import BeautifulSoup

from PyKomoran import *

komoran = Komoran(DEFAULT_MODEL['FULL'])
komoran.set_user_dic('D:/petition/user_dic.dic')
# komoran.setUserdic('D:/petition/user_dic.dic')

def komoran_tokenizer(sent):
    words = komoran.get_list(sent)
    words = [str(w) for w in words if ('/NN' in str(w) or '/XR' in str(w) or '/VA' in str(w) or '/VV' in str(w))]
    return ' '.join([word.split("/")[0] for word in words])

def komoran_tokenizer(sent):
    words = komoran.get_list(sent)
    words = [str(w) for w in words if ('/NN' in str(w) in str(w))]
    return ' '.join([word.split("/")[0] for word in words])


data = pd.read_pickle("../../petition/petition_total_data.pkl")
data_add = pd.read_pickle("../../petition/total_data.pkl")
data = pd.concat([data,data_add], axis = 0)
data = data.reset_index(drop=True)
data['progress'] = data['progress'].map(lambda x: x.replace("\n",""))
data.shape

(40667, 10)

In [2]:
data['sdays'] = pd.to_datetime(data['sdays'])
data['edays'] = pd.to_datetime(data['edays'])

data['count'] = pd.to_numeric(data['count'])

# 2019.03.31 이후 sns 계정에서 100명 이상의 동의를 받은 후에만 청원 글 게시 가능 

In [3]:
print(f" 가장 많은 동의 수 {data['count'].max()}, 가장 적은 동의 수 {data['count'].min()}")

 가장 많은 동의 수 2715626.0, 가장 적은 동의 수 0.0


In [4]:
data = data[data['sdays']>=datetime.strptime("2019.04.01", "%Y.%m.%d")]
data = data[data['sdays']<datetime.strptime("2021.03.31", "%Y.%m.%d")]

print(f" 가장 많은 동의 수 {data['count'].max()}, 가장 적은 동의 수 {data['count'].min()}")

 가장 많은 동의 수 2715626.0, 가장 적은 동의 수 107.0


In [5]:
data.shape

(17811, 10)

In [6]:
data.columns

Index(['code', 'sdays', 'edays', 'title', 'count', 'content', 'category',
       'progress', 'link', 'person'],
      dtype='object')

In [7]:
data['progress'].value_counts()  # 답변완료 : 127

청원종료      17479
청원진행중       196
답변완료        136
Name: progress, dtype: int64

In [8]:
data = data.drop_duplicates(['title'])

In [9]:
data.shape

(17751, 10)

In [10]:
print(f" 가장 많은 동의 수 {data['count'].max()}, 가장 적은 동의 수 {data['count'].min()}")

 가장 많은 동의 수 2715626.0, 가장 적은 동의 수 107.0


In [11]:
data = data.drop_duplicates("title")
data.shape

(17751, 10)

In [12]:
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
MULTIPLE_SPACE = re.compile(" +", re.UNICODE)
EMOJI_PATTERN = re.compile("["
        u"\U0001F000-\U0001FFFF"  # 1F000~1FFFF 추가 
        u"\U000F0850-\U000F0853"               "]+", flags=re.UNICODE)

ETC_PATTERN = re.compile('[-=+,#/\?:^$@*\"●<>☆!★()▲▼■※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', re.UNICODE)
ETC_PATTERN2 = ['\n','\t+','\r', r"\r\n", '청와대','안녕하세요', "감사합니다",u'\xa0',u'\x0b',u'\\xa0']
ETC_PATTERN3 = "본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다"

### 맞춤법 검사

In [13]:
from hanspell import spell_checker

In [14]:
def spell_check(text):
    
    text_lst = []

    if len(text) >= 500:
        i =0
        j = 500

        while i < len(text):


            text_lst.append(spell_checker.check(text[i:j]).as_dict()["checked"])

            i += 500
            j += 500

    else:
        
        text_lst = spell_checker.check(text).as_dict()["checked"]
        
    return ''.join(text_lst)


In [15]:
def preprocessing_content(content_series):
    content_series = [EMAIL_PATTERN.sub(r' ', content) for content in content_series]
    content_series = [URL_PATTERN.sub(r' ', content) for content in content_series]

    content_series = [EMOJI_PATTERN.sub(r' ', content) for content in content_series]
    content_series = [ETC_PATTERN.sub(r' ', content) for content in content_series]

    content_series = [re.sub('|'.join(ETC_PATTERN2),u' ', content).replace(ETC_PATTERN3,"") for content in content_series]
    content_series = [MULTIPLE_SPACE.sub(r' ', content) for content in content_series]

    return content_series

In [16]:
data['pre_content'] = preprocessing_content(data['content'])

In [17]:
data = data[~data['content'].isin(['반대합니다','찬성합니다','동의합니다','동참합니다'])]

In [18]:
data = data.reset_index(drop=True)

In [19]:
data["pre_content"] = data['pre_content'].map(lambda x:x.replace("\\xa0"," "))

In [20]:
data.shape

(17751, 11)

In [21]:
data = data.reset_index()

In [158]:
import time
import random

# lst = []


for i in range(len(data)):
    lst.append(spell_check(data['pre_content'][i].replace("\x1c","")))

    if i%40==0:
        time.sleep(random.uniform(5,7))
        print(i)

15120
15160
15200
15240
15280
15320
15360
15400
15440
15480
15520
15560
15600
15640
15680
15720
15760
15800
15840
15880
15920
15960
16000
16040
16080
16120
16160
16200
16240
16280
16320
16360
16400
16440
16480
16520
16560
16600
16640
16680
16720
16760
16800
16840
16880
16920
16960
17000
17040
17080
17120
17160
17200
17240
17280
17320
17360
17400
17440
17480
17520
17560
17600
17640
17680
17720


In [159]:
print(len(lst)) # 13951
print(len(data))

17751
17751


In [162]:
data['pre_content'] = lst

In [163]:
data["prior_content"] = data['pre_content'].map(lambda x : x.split(".")[0] + x.split(".")[len(x.split("."))-1])
data["for_content"] = data['pre_content'].map(lambda x : ' '.join(x.split(".")[1:len(x.split("."))-2]))

In [164]:
data.to_pickle("end_hanspell.pkl")  # 데이터 저장 

### 본문 요약
* gensim을 이용하여 본문을 요약하는데 글의 첫 줄과 마지막 줄이 주로 주장하는 문장이기 때문에 첫줄+요약글+마지막글로 저장

In [165]:
data = data.reset_index(drop =True)

data['summary_content'] = ''

count = 1

for i in range(len(data)):
    
    # 본문이 특정 개수 문장 이하일 경우 결과가 반환되지 않음.
    # 이때는 요약하지 않고 본문에서 앞 3문장을 사용함.
        
    try:
        
        data['summary_content'][i] = summarize(data['for_content'][i], word_count= 0.15)
        
    except ValueError:
        
        data['summary_content'][i] = None
        
    if not data['summary_content'][i]:
        
        df_sentences =  data['for_content'][i].split('.')

        if len(df_sentences) > 3:

            data['']
            data['summary_content'][i] = '.'.join(df_sentences[:3])
            

        else:
            data['summary_content'][i] = '.'.join(df_sentences)
            
            count += 1

In [166]:
data["summary_content_end"] = data['prior_content'] + data['summary_content'] + data['title']

In [167]:
for i in range(10):
    print("\n")
    print(data["summary_content_end"][i])
    print("--"*10)



“정부 아이돌봄서비스 아이돌보미 영유아 폭행 강력 처벌 및 재발방지 방안 수립을 부탁합니다”  저희는 정부에서 제공하는 아이돌봄서비스 이용 중입니다  금천구에 살면서 14개월 아기를 키우고 있는 맞벌이 부부입니다  건강보험료 부부합산 기준치 초과되는 이유로 아무 지원받지 못하는 라 형을 이용하고 있었습니다  어떠한 지원도 없었지만 정부에서 소개해주는 돌보미 선생님이기에 믿고 이용하였습니다  하지만 14개월이 된 저희 아이를 약 3개월 넘도록 지속적으로 학대하고 있었음을 CCTV를 통해 확인하게 되었습니다  폭행 영상은 아래의 링크를 통해 확인해 주시길 바랍니다  폭행 내용은 다음과 같습니다  따귀와 딱 밥을 때리고 아이가 아파서 울면 우는 입에 밥을 밀어 넣기도 하고 밥 먹다 아기가 재채기를 하면 밥풀이 튀었다는 이유로 아이를 때리고 소리 지르며 꼬집기도 했습니다  그뿐만 아니라 아기가 자는 방에서 뒤통수를 때리고 머리채를 잡고 발로 차고 따귀를 때리는 등 갖가지 폭언과 폭행들이 확인되었습니다  현재 저희 부부에게 사과문을 전달한 아이돌보미는 저희 부부를 위해 그리고 아이를 위해 그랬다고 합니다  그리고 이번 일로 자신은 해고를 당하였고 6년의 노고가 물거품이 되었다고 합니다  저 말도 너무 화가 났지만 저희 아이를 이 정도까지 학대한 사람이 6년이나 아이 돌봄 선생님으로 활동을 했다는 게 정말 너무 무섭고 소름이 끼칩니다  조금이라도 늦게 발견했다면 아이에게 큰일이 일어날 수도 있었을 사건이었습니다  3개월이라는 기간 동안 말도 못 하고 학대를 견뎌야만 했을 14개월이 된 아이를 생각하면 그저 눈물만 납니다  아이돌봄서비스는 돌봄이 필요한 가정에 찾아가는 시간제 서비스로 1 1로 아동을 안전하게 돌보는 서비스로 사이트 내 소개되어 있을 뿐만 아니라 수많은 맞벌이 부부들이 사용하고 있는 정부 지원 서비스입니다  하지만 아이돌봄서비스를 직접 이용해본 저로서는 아기의 안전을 보장해주기에는 너무 부실한 부분들이 많았습니다  관련하여 아래와 같은 제도적 보완이 꼭

In [168]:
data.to_pickle("clean_data_for_sentiment_classification.pkl")

### 청원진행중인 게시글의 유사도 검정

In [169]:
data = data[data['progress']=='청원진행중 ']

In [170]:
doc_nouns_list = [komoran_tokenizer(doc) for doc in data['summary_content_end']]

tfidf_vectorizer = TfidfVectorizer(min_df=3,stop_words=['지금',"요약","인지","다기","부분","드리","일부","기간","가져오","대통령","청원","해주","통하","후에","지내","줄이","이나","위하","등등","시키","나오",'따르','인해','때문','문제','당장','청원','청와대'])  # 한 번만 나타나는 단어들은 무시
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

words = np.array(tfidf_vectorizer.get_feature_names())  # 단어 추출 

id2vocab = {tfidf_vectorizer.vocabulary_[token]:token for token in tfidf_vectorizer.vocabulary_.keys()}
curr_doc, result = tfidf_matrix[0],[]

for idx, el in zip(curr_doc.indices, curr_doc.data):
    result.append((id2vocab[idx],el))

In [171]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 60)
vecs = svd.fit_transform(tfidf_matrix)

In [174]:
topn = 5

data = data.reset_index(drop=True)

dic = {}

for i in range(20):
    query_doc_vec = vecs[i]
    query_vec_norm = np.linalg.norm(query_doc_vec)
    query_unit_vec = query_doc_vec / query_vec_norm
    
    query_sentence = data.title[i]
    scores = np.dot(vecs, query_unit_vec)
    dic[query_sentence] = sorted(zip(data.title.values.tolist(), scores), key=lambda x: x[1], reverse=True)[1:topn + 1]

In [175]:
dic

{'코로나19 거리두기 체계 개선해주세요 (코로나19 전쟁의 승리)': [('엄벌해주세요', 0.6250651931278408),
  ('대중음악 공연(콘서트) 방역지침을 변경해 주세요.', 0.5794402944966173),
  ('자영업을 두번 죽이는 새로운 거리두기 개편안을 반대합니다!!', 0.5265032691400833),
  ('정부의 코로나19방역 수칙과 반대로 직원들을 한군데 집합시켜 업무하게 하는 회사 정책을 늦춰주세요.',
   0.5130269777858942),
  ('4차 재난지원금 제한업종의 매출 비교 지원은 위헌이며 현실을 모르는 탁상행정의 끝입니다.', 0.21920107375289274)],
 '철도공사하면 멀쩡하던 주택도 노후주택이 된다네요!!!': [('부동산(주택)문제 해결방법   제안자:  *** (2021.3.16.)',
   0.4622648064394955),
  ('박상혁의원 외 15인 국회의원이 발의한 빈집 및 소규모 주택 정비에 관한 특례법 일부개정법률안 (의안번호 8637) 즉시 철회바랍니다.',
   0.445872689019026),
  ('LH는 신혼희망타운 분양세대 건축비로 행복주택 건축하는 부분을 정확히 설명해라!!', 0.3667302704303826),
  ('**초교 앞 지하철 공사 착굴(환기구)위치 변경 요청', 0.3599714696196599),
  ('인테리어사기!! 강력한 처벌법 만들어주세요??', 0.3210918133454811)],
 '기계설비유지관리자  5년 유예기간 (기존사업소 ) 시행력 철거 요구': [('기계설비유지관리자 관련 기계설비법 시행규칙 부칙 제2조 제2항의 철회를 요청합니다.',
   0.9085215375236623),
  ('탈원전 정책은 국민을 속이고 국가를 망치는 범죄행위입니다.', 0.16077945601367355),
  ('경남 개발공사, ** *** 아파트의 일방적인 계약내용 변경', 0.12432769491317451),
  ('농아인 성폭력범죄

In [176]:
from sklearn.decomposition import NMF

vector_array = tfidf_matrix.toarray()
nmf = NMF(n_components = 38, random_state = 20171490)  # 군집화
nmf.fit(vector_array)

features = nmf.transform(vector_array)

normalizer = Normalizer()
norm_features=normalizer.fit_transform(features)

df_features = pd.DataFrame(norm_features,index=data['title'].tolist())

In [177]:
components_df = pd.DataFrame(nmf.components_, columns=words)

In [178]:
topic_lst = []
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    topic_df = pd.DataFrame(tmp.nlargest(10))
    topic_df.columns = ['value']
    topic_lst.append(topic_df)

In [179]:
topic_lst

[       value
 학교  5.643140
 학생  4.861940
 교수  3.970195
 사실  1.429848
 학년  1.347457
 학과  1.286904
 직장  1.208808
 선택  1.165551
 조사  1.045875
 관계  1.013278,
         value
 역사   0.760812
 드라마  0.726029
 왜곡   0.356314
 방영   0.257753
 작가   0.242605
 마사   0.227222
 중국   0.213100
 인물   0.171815
 장면   0.146137
 모독   0.122396,
          value
 영업    2.786689
 제한    2.500635
 매출    2.135089
 지원금   1.887523
 소상공인  1.349515
 버팀목   1.189792
 재난    1.111757
 플러스   1.057210
 년대    0.703694
 자금    0.648614,
        value
 병원  0.888661
 치료  0.818685
 환자  0.520479
 이식  0.514322
 입원  0.452427
 복무  0.390918
 군대  0.388362
 아들  0.371653
 전역  0.315165
 항암  0.300618,
          value
 교사    1.807938
 보육    1.065466
 검사    0.757409
 아이들   0.498443
 어린이집  0.476569
 시간    0.364231
 코로나   0.356068
 학부모   0.248412
 근무    0.237144
 조심    0.223778,
         value
 분양   1.481401
 임대   1.035750
 주택   0.697262
 아파트  0.546571
 세대   0.382322
 조기   0.361672
 임차인  0.333635
 입주   0.297736
 사업자  0.294032
 상환   0.288191,
    

In [183]:
df_features = pd.DataFrame(norm_features,index=data['title'].tolist())

def similarity_print(article_title):
    article_category = data[data['title']==article_title]['category'].values
    print("# ", article_title, article_category )
    article = df_features.loc[article_title]

    similarities = df_features.dot(article)
    top = similarities.nlargest(3)

    count = 1

    for i in range(1, len(top)):
        
        if top[i] > 0.8:
            similar_article_category = data[data['title']==top.index[i]]['category'].values

            print(f"{count}: "+top.index[i]+'\n Similarities'+str(top[i]))

            count+=1 

    if count==1:    

        print('유사한 청원이 없습니다.')

In [184]:
title_lst = data[data['progress']=='청원진행중 '].title.values.tolist()

In [185]:
for title in title_lst:
    article_title = title
    similarity_print(article_title)

#  코로나19 거리두기 체계 개선해주세요 (코로나19 전쟁의 승리) ['보건복지']
1: 대중음악 공연(콘서트) 방역지침을 변경해 주세요.
 Similarities0.9976850144730621
2: 엄벌해주세요
 Similarities0.9970374467850968
#  철도공사하면 멀쩡하던 주택도 노후주택이 된다네요!!! ['교통/건축/국토']
1: 인천공항공사에서 캐디들의 일자리를 없애려고 합니다.
 Similarities0.952875683876272
#  기계설비유지관리자  5년 유예기간 (기존사업소 ) 시행력 철거 요구 ['일자리']
1: 기계설비유지관리자 관련 기계설비법 시행규칙 부칙 제2조 제2항의 철회를 요청합니다.
 Similarities0.999269751706341
#  금감원의 주식 자전거래 감독 강화를 요청드립니다 ['경제민주화']
1: **글로벌 청원합니다.
 Similarities0.9857398479513018
2: ■ 한개라도, 제대로 해 놓고 가시라 .
 Similarities0.9482347406658027
#  LH와 관련된 탐관오리들을 처벌해 주시고 재산은 국고에 환수하길 원합니다. ['기타']
1: 독도는 우리땅을 독도는 대한민국 땅 으로 표현을 바꿔주세요
 Similarities0.9898228963562039
2: 현역 대한민국 사관생도가 우국충정으로 대통령님께 고언을 올립니다.
 Similarities0.97659976282261
#  '정치부기자의 폭행으로 인한 실명사건'가해자의 부인으로서 사건의 진실에 대해 알려드리고자 합니다. ['인권/성평등']
1: 보이스피싱범죄의 전달책에게 강한 처벌을 간곡히 호소드려요
 Similarities0.9882869786647462
2: 노원 일가족 3명 살인사건의 가해자 20대 남성 신상공개 촉구 바랍니다.
 Similarities0.9782755772723438
#  조상의 묘를 무분별하게 파헤치고 훼손하는 육촌 이웃을 저지하는 것을 도와주십시오! ['기타']
1: 층

#  인천시의 횡포를 막아주세요 ['정치개혁']
1: 50년간 살아온 장애인 가족의 집을 포항시는 강제로 뺐어야만 하나요.?
 Similarities0.9804272446007276
2: 우리 지역의 명품공원 예산이 시장의 의사로 축소 되었기에 청원합니다
 Similarities0.9623288335153152
#  병원입원시 코로나검사비용 환자.보호자비용 절감부탁드립니다 ['보건복지']
유사한 청원이 없습니다.
#  SBS 월화 드라마 xx구마사 방영 중단 ['문화/예술/체육/언론']
1: 역사왜곡 동북공정 드라마 < **구마사 >의 즉각 방영중지를 요청합니다.
 Similarities0.9875838361280642
2: 드라마 작가 ***과 관계자들을 국가모독죄, 허위사실유포죄, 인격모독죄, 명예회손죄 등으로 처벌해주세요.
 Similarities0.9871606698411397
#  역사왜곡 동북공정 드라마 < **구마사 >의 즉각 방영중지를 요청합니다. ['문화/예술/체육/언론']
1: 드라마 작가 ***과 관계자들을 국가모독죄, 허위사실유포죄, 인격모독죄, 명예회손죄 등으로 처벌해주세요.
 Similarities0.9965422462248397
2: JTBC 신작 드라마 설**에 대하여 방송금지 시켜 주시기 바랍니다.
 Similarities0.9884696633827942
#  어린이보호차량 관련 차량검사 강화에 대하여,,,, ['안전/환경']
1: 어린이집 차량 썬팅을 벗기지 않길 바랍니다.
 Similarities0.9835267682209479
2: 어린이 운송용 차량의 썬팅 단속을 하지말아 주세요!!
 Similarities0.9763308852701197
#  역사 왜곡 및 문화 왜곡 방송물에 대한 법적 제재 요청드립니다. ['문화/예술/체육/언론']
유사한 청원이 없습니다.
#  **초교 앞 지하철 공사 착굴(환기구)위치 변경 요청 ['안전/환경']
유사한 청원이 없습니다.
#  대통령의 사진까지 조작하여 국제사기에 이용하는 국제사기꾼에

 Similarities0.871416507054958
2: 저는 뉴스에 나왔던 2020년 LH 투기의 최초 제보자 입니다.
 Similarities0.8538342002937351
#  금소법 6대판매원칙중 광고에 대한 부분이 불합리적입니다 ['일자리']
유사한 청원이 없습니다.
#  **전자 **사업부 중국청도법인의 임가공협력사에 대한 업무 처리 ['경제민주화']
1: 기업의 생사여탈권을 가진 갑질 먹튀 회계법인의 무책임함을 바로 잡아주세요.
 Similarities0.9876070014131502
2: 근로기준법위반에 대한 법 개정 청원
 Similarities0.9710342969063656
#  탈원전 정책은 국민을 속이고 국가를 망치는 범죄행위입니다. ['성장동력']
1: 전기요금 연료비 연동제대로 실시해주세요!!!
 Similarities0.9929056017661152
2: 국가 반도체 핵심기술 유출방지를 위한 매그나칩반도체의 중국자본 매각을 막아주십시오!
 Similarities0.8477437053838434
#  LH 조기분양 불법행위를 처벌해주세요 ['교통/건축/국토']
1: 공공지원 민간임대(5년) 아파트 분양전환 관련, 호소문
 Similarities0.995847749718149
2: LH는 신혼희망타운 분양세대 건축비로 행복주택 건축하는 부분을 정확히 설명해라!!
 Similarities0.9703339976992089
#  검찰의 개인정보 유출 및 불법 사용에 대한 민원 ['행정']
1: 인권을 유린하고 무리한 수사로 억울한 사람을 만든 경찰수사를 규탄합니다.
 Similarities0.9343504239740997
2: 경찰간부 갑질에 매일매일 분신자살하고 싶다는 생각을 합니다.  너무억울합니다. 살려주세요
 Similarities0.8635636000472788
#  아동 발달 재활 서비스 예산 확대 ['육아/교육']
1: 억울하게 뺏긴 제 아이를 찾아주세요
 Similarities0.9069767634638624
2: 반려동

 Similarities0.9581008855251892
2: 4차 재난 지원금 형평성... 이게 맞습니까?
 Similarities0.9052283381475857
#  백혈병환자가족에게 코로나19백신 최우선접종을 청원합니다. ['보건복지']
1: 킴리아의 빠른 건강 보험 적용을 촉구합니다
 Similarities0.9569658123936141
2: 왜? 군 복무 중 발병한 난치병을 개인책임으로 떠넘깁니까?
 Similarities0.9531862610081062
#  한복을 우리나라 문화재로 지정해주세요 ['문화/예술/체육/언론']
1: 강원도의 중국복합문화타운 건설을 막아야 합니다.
 Similarities0.9360061415864899
2: 강원도 차이나타운 건설을 철회해주세요.
 Similarities0.9167772696977052


LSA 의 성능이 더 좋음

### nmf cross validation

In [341]:
'''
from surprise.model_selection import cross_validate

algo = NMF(n_components=35, solver="mu")
cv_results = cross_validate(algo, tfidf_matrix, measures=['RMSE'], cv=5, verbose=False)
'''

'\nfrom surprise.model_selection import cross_validate\n\nalgo = NMF(n_components=35, solver="mu")\ncv_results = cross_validate(algo, tfidf_matrix, measures=[\'RMSE\'], cv=5, verbose=False)\n'

In [316]:
'''
# Create a DataFrame: components_df
components_df = pd.DataFrame(nmf.components_, columns=words)

topic_lst = []
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    topic_df = pd.DataFrame(tmp.nlargest(10))
    topic_df.columns = ['value']
    topic_lst.append(topic_df)
    
topic_lst
'''

### PageRank

In [653]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.komoran = komoran
        self.stopwords = ['대통령', "청와대","아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희"]
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.komoran.nouns(str(sentence))
                if noun not in self.stopwords and len(noun) > 1]))
        
        return nouns


In [654]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
            self.nouns = self.sent_tokenize.get_nouns(self.sentences)
            self.graph_matrix = GraphMatrix()
            self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
            self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
            self.rank = Rank()
            self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
            self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
            self.word_rank_idx = self.rank.get_ranks(self.words_graph)
            self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]

        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
            index.sort()

        for idx in index:
            summary.append(self.sentences[idx])

        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]

        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])

        return keywords

In [655]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}


In [656]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}


In [657]:
data['similar_content'] = ''

for i in range(len(data)):
    textrank = TextRank(data.content[i])
    
    data['similar_content'][i] = ''.join(textrank.summarize(3))
    
    if data['similar_content'][i] == '':
        data['similar_content'][i] = data['content']

In [658]:
data['summary_content'] = data['title']+data['similar_content']

In [659]:
import json
import urllib
from bs4 import BeautifulSoup

def spellchecker(q):

    params = urllib.parse.urlencode({
        "_callback": "",
        "q": q
    })
    
    # 네이버 맞춤법 검사기 사용하여 문법 교정 
    data = urllib.request.urlopen("https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn?" + params)
    data = data.read().decode("utf-8")[1:-2]
    data = json.loads(data)
    data = data["message"]["result"]["html"]
    data = BeautifulSoup(data, "html.parser").getText()
    
    return data

In [660]:
data["summary_content"] = data["summary_content"].map(lambda x: x.replace("\\",""))
data["summary_content"] = data["summary_content"].map(lambda x: spellchecker(re.sub(r'[0-9]+', '',cleanText(x))))

In [661]:
doc_nouns_list = [komoran_tokenizer(doc) for doc in data['summary_content']]

tfidf_vectorizer = TfidfVectorizer(min_df=3,stop_words=['지금',"요약","인지","다기","부분","드리","일부","기간","가져오","대통령","청원","해주","통하","후에","지내","줄이","이나","위하","등등","시키","나오",'따르','인해','때문','문제','당장','청원','청와대'])  # 한 번만 나타나는 단어들은 무시
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

words = np.array(tfidf_vectorizer.get_feature_names())  # 단어 추출 

id2vocab = {tfidf_vectorizer.vocabulary_[token]:token for token in tfidf_vectorizer.vocabulary_.keys()}
curr_doc, result = tfidf_matrix[0],[]

for idx, el in zip(curr_doc.indices, curr_doc.data):
    result.append((id2vocab[idx],el))

In [662]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 100)
vecs = svd.fit_transform(tfidf_matrix)

In [663]:
topn = 5

dic = {}

for i in range(20):
    query_doc_vec = vecs[i]
    query_vec_norm = np.linalg.norm(query_doc_vec)
    query_unit_vec = query_doc_vec / query_vec_norm
    
    query_sentence = data.title[i]
    scores = np.dot(vecs, query_unit_vec)
    
    sim = sorted(zip(data.title.values.tolist(), scores), key=lambda x: x[1], reverse=True)[1:topn + 1]
    
    sim_lst = []
    
    for j in range(topn):
        if sim[j][1] >= 0.35:
            sim_lst.append(sim[j])
        
    dic[query_sentence] = sim_lst

In [664]:
dic

{'코로나19 거리두기 체계 개선해주세요 (코로나19 전쟁의 승리)': [('대중음악 공연(콘서트) 방역지침을 변경해 주세요.',
   0.7276953924055376),
  ('엄벌해주세요', 0.5293947123294707),
  ('정부의 코로나19방역 수칙과 반대로 직원들을 한군데 집합시켜 업무하게 하는 회사 정책을 늦춰주세요.',
   0.4981791126261499)],
 '철도공사하면 멀쩡하던 주택도 노후주택이 된다네요!!!': [('"네 발 뭉개져 피투성이" 개 매달고 달린 ***',
   0.3986236004225412),
  ('과천 3기신도시 주택 사기분양 업자를 고발합니다', 0.39652555199094086),
  ('박상혁의원 외 15인 국회의원이 발의한 빈집 및 소규모 주택 정비에 관한 특례법 일부개정법률안 (의안번호 8637) 즉시 철회바랍니다.',
   0.35932052203299436),
  ('부동산(주택)문제 해결방법   제안자:  *** (2021.3.16.)', 0.35628945737697604)],
 '기계설비유지관리자  5년 유예기간 (기존사업소 ) 시행력 철거 요구': [('기계설비유지관리자 관련 기계설비법 시행규칙 부칙 제2조 제2항의 철회를 요청합니다.',
   0.827680637934771)],
 '금감원의 주식 자전거래 감독 강화를 요청드립니다': [],
 'LH와 관련된 탐관오리들을 처벌해 주시고 재산은 국고에 환수하길 원합니다.': [('16개월입양아 사망사건에 일조하고 징계에 불복하는 뻔뻔한 경찰관들을 파면 해주세요.',
   0.43955954391265417),
  ('“광주 민간공원 특례사업은 LH 사태 확대판인가?” (공직자 부동산 투기 의혹 진정서)', 0.39529594086938663)],
 "'정치부기자의 폭행으로 인한 실명사건'가해자의 부인으로서 사건의 진실에 대해 알려드리고자 합니다.": [('노원 일가족 3명 살인사건의 가해자 20대 남성 신상공개 촉구 바랍니다.',
 

In [578]:
'''
import networkx
import re

class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            # print("line : ", line)
            # str로 들어온 문장을 특수문자와 일반문자를 구분하여 list형태로 저장 
            ch = self.rgxSplitter.split(line)
            # print("ch : ", ch)
            # print("ch[::2]: ",ch[::2], "ch[1::2]:", ch[1::2])
            # 문장과 점을 이어붙여 완전한 문장의 형태로 만들고 반환함. 
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                # print("map s : ", s)
                if not s: continue
                yield s
                
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 7)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        # Similarity
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: 
                    s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            #해당 문장을 토크나이저로 자른 형태들, 2보다 작다면 이는 여기서 NNG, NN, VV, VA을 포함하는 요소가 아예 없거나 하나밖에 없다는 뜻
            if len(s) < 2: 
                continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
            #sentSet : {('아버지', 'NNG'), ('식당', 'NNG')} 등의 형태로 문장의 토큰들을 저장한 곳.

        #모든 문장의 조합에 대해서 similarity 계산 후 dicBiCount에 저장.
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 

    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
  
  
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)
        score = int(len(r)*ratio)

        if score < 5 : 
            score = len(r) 
        elif score >= 5:
            score = 5
        else:
            pass
        
        ks = ks[:score]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))
        
'''

In [595]:
import networkx
import re
from konlpy.tag import Kkma

class RawSentence:
    def __init__(self, textIter):
        self.kkma = Kkma
        
        if type(textIter) == str: 
            self.textIter =  self.kkma.sentences(textIter)
            
        else: self.textIter = textIter
            
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s
 
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)
 
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
 
 
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)
        score = int(len(r)*ratio)

        if score < 5 : 
            score = len(r) 
        elif score >= 5:
            score = 5
        else:
            pass
        
        ks = ks[:score]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))
